# Objectives 

Auction item performance
- before/after, sort of recommendations for aution item listing
- suggest bidding increment based on starting price
- performance score after auction (high activity, high ending price vs starting price, large delta between value and winning bid)
- account for traffic and engagement levels (low traffic/engagement auctions shouldn't punish items)
- compelling reason to force the user to enter item value prior to starting price?

# Process

- need to qualify successful vs unsuccessful auctions and/or items
    - start with score objective in order to easily identify
    - performance needs to be tuned to traffic
- identify settings to be recommended
    - starting price vs value
    - bidding increment
    
# Sources

- auctionitem: bid increment, value, reserve
- bidders: bids
- transauction: purchase price

In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *
%matplotlib inline

In [3]:
# product, # of bids, bid increment, value, start $, end $
q = "select * from auctionitem"
df_ai = redshift_query_read(q, schema='production')

In [17]:
q = '''select
            ta.product,
            ta.total as price,
            ta.status as status,
            count(distinct(b.ticketholder)) as bidders,
            count(b.ticketholder) as bids,
            count(distinct(ta.id)) as transactions,
            count(distinct(ta.bidder)) as winners
        from bidders as b
            left join transauction as ta on b.product=ta.product
        group by ta.product, ta.total, ta.status'''
df_bids = redshift_query_read(q, schema='production')

In [18]:
df = df_ai.drop('id', axis=1).merge(df_bids, on='product')

In [25]:
df.tail(3)

,product,winner,maxbid,reserve,bidincrement,number,value,price,status,bidders,bids,transactions,winners
23789,112171,40371,200.0,0.0,5.0,40,175.0,70.0,1.0,1,11,1,1
23790,116897,45855,0.0,0.0,10.0,805,250.0,270.0,1.0,1,19,1,1
23791,129923,69546,0.0,0.0,150.0,192,3056.0,250.0,1.0,1,1,1,1


We see a bids excceeding bidders because anonymous bids are logged as 0 and therefore grouped as a single bidder in the numbers. We have to presume that mulitple bids without an increase in bidders represents distinct individuals.

In [24]:
print("Number of items: {:,}".format(len(df)))
print("Items value > $0: {:,}".format(len(df[df['value']>0])))
print("Items price > value: {:,}".format(len(df[df['price']>df['value']])))
print("Items w/ > 1 bidders (considering anonymous): {:,}".format(len(df[df['bidders']>1])))
print("Items w/ > 1 bids: {:,}".format(len(df[df['bids']>1])))

len_items = len(df[(df['value']>0)&(df['bids']>1)])
print("Items w/ value > $0 and > 1 bidders: {:,}".format(len_items))

Number of items: 23,792
Items value > $0: 18,004
Items price > value: 9,375
Items w/ > 1 bidders (considering anonymous): 560
Items w/ > 1 bids: 19,241
Items w/ value > $0 and > 1 bidders: 14,664


__Restrict data to (1) items with value > $0 and (2) > 1 bids.__ This will elliminate items that can't be compared to a base value of the item being auctioned and those that failed to gain real engagement.